<img src='sql-alchemy-logo.jpg'>

# O que é o SQLAlchemy ?

SQLAlchemy é uma biblioteca de mapeamento objeto-relacional SQL em código aberto desenvolvido para a linguagem de programação Python e disponibilizado sobre a licença MIT.

O SQLAlchemy fornece "um conjunto completo e bem conhecido de modelos de persistência de nível corporativo, projetado para eficiência e alta performance de acesso a banco de dados, adaptado em uma linguagem de domínio simples e Pytônica". Sua filosofia é que bancos de dados SQL se comportem cada vez menos como coleções de objetos, em que mais tamanho e performance comecem a importar, enquanto coleções de objeto se comportem cada vez menos como tabelas e linhas, no qual mais abstração começa a importar. Por esta razão o SQLAlchemy adotou o padrão mapeador de dados (como o Hibernate para Java) em vez do padrão de registro ativo usado por vários outros mapeadores objeto-relacional. Entretanto, plugins adicionais como Elixir e declarative permitem aos usuários desenvolverem usando sintaxe declarativa.

O SQLAlchemy foi lançado em fevereiro de 2006 e rapidamente se tornou uma das ferramentas de mapeamento objeto-relacional mais utilizadas juntamente com o ORM do Django, na comunidade Python.

# Qual o objetivo desse tutorial?

Ensinar a como criar uma tabela em um banco de dados para guardar os dados de um determinado dataset e consultar dados do mesmo usando o sqlalchemy.

# Baixando SQLAlchemy

Para linux, execute o seguinte comando no seu terminal:

```sudo pip install sqlalchemy```

# Fazendo o Load de nosso Dataset

Para fins didáticos vamos usar o dataset do sklearn chamado Iris. Caso não conheça esse dataset e deseje saber mais sobre os dados e história desse dataset clique [aqui](https://en.wikipedia.org/wiki/Iris_flower_data_set).

In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [2]:
iris = datasets.load_iris()

In [3]:
# vamos transformá-lo em uma dataframe pandas 
# para facilitar o manuseio
iris = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [4]:
# vamos dar uma olhada na cara dele
iris.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0


# Criando conexão com nosso Banco de Dados

<img src='engine.png'>

Partindo do presuposto que você já tenha seu banco de dados, quer em sua máquina local ou algum servidor remoto, você deverá se conectar com o mesmo por meio de uma engine do SQLAlchemy usando o seguinte url:

    '{TipoDoBancoDeDados}://{NomeUsuario}:{SenhaDoBanco}@{HostDoBanco}/{NomeDoBanco}
    
A seuir estão listados alguns tipos de Banco de Dados que o SQLAlchemy consegue estabelecer conexão e seus respectivos valores a serem passados no TipoDoBancoDeDados da url acima:

* Firebird --> firebird
* Microsoft SQL Server --> mssql
* MySQL --> mysql
* Oracle --> oracle
* PostgreSQL --> postgresql
* SQLite --> sqlite

É uma boa prática de segurança nunca passar senhas, hots, usuários ou coisas do tipo, explicitamente em seu código. Por isso, crie variáveis ambientes em uma sessão de seu terminal executando os seguintes comandos conforme imagem abaixo:

<img src='variaveis_ambiente.png'>

Portanto, antes de executar o código da próxima célula, defina suas variáveis ambiente com os respectivos valores do seu Banco de Dados. Se seu banco estiver na sua máquina local, então HOST_DO_MEU_DB='localhost'. 

In [5]:
import os # essa biblioteca será usada para acessar as variáveis ambientes que criei

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# coletando os valores das minhas variáveis ambientes
DB = os.environ["NOME_DO_MEU_DB"] 
HOST = os.environ["HOST_DO_MEU_DB"] 
USER = os.environ["USUARIO_DO_MEU_DB"]
PASS = os.environ["SENHA_DO_MEU_DB"] 

# neste caso, como meu Banco de Dados é do 
# tipo Postgre eu defini diretamente na url de conexão
# o TipoDoBancoDeDados como postgresql
engine = create_engine('postgresql://{}:{}@{}/{}'.format(USER, PASS, HOST, DB))
Base = declarative_base()
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

# função para criar a conexão com o banco
def create_session(USER, PASS, HOST, DB):
    engine = create_engine('postgresql://{}:{}@{}/{}'.format(USER, PASS, HOST, DB))
    Base = declarative_base()
    Base.metadata.bind = engine
    DBSession = sessionmaker(bind=engine)
    session = DBSession()
    return session

# Criando nosso Objeto Tabela

Vamos criar uma classe que irá representar nossa tabela que será criada lá em nosso banco de dados. Dei o nome de **iris** para minha tabela e ela será criada em um *schema* que eu já criei em meu banco postgre chamado **api**. 

In [6]:
from sqlalchemy import Column, Float, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base() 

class IRIS(Base):
    __tablename__ = 'iris'
    __table_args__ = {'schema':'api'}
    
    # Em nosso dataset, todos as colunas armazenam dados do tipo float
    # então iremos declar que cada uma dessas variáveis é uma
    # coluna e o tipo de dados que ela armazena, no caso float.
    # Se nossa coluna armazenasse dados do tipo String poderíamos
    # definir o tamanho máximo dessas strings. Ex:
    # tipo_de_transacao = Column(String(30))
    # Para declarar uma coluna que receberá data, ou
    # inteiros muito grandes podemos importar do sqlalchemy
    # os respectivos objetos DateTime e BigInteger.
    
    # como declarei que essa coluna é uma primary_key, 
    # isso indica que ela não pode receber valores repetidos.
    # Pelo menos uma das colunas deve ser primary_key.
    ids = Column(Integer, primary_key=True)
    sepal_length = Column(Float)
    sepal_width = Column(Float)
    petal_length = Column(Float)
    petal_width = Column(Float)
    target = Column(Float)
    
# Cria a tabela no engine. Esse comando é equivalente
# ao "Create Table" do SQL.
Base.metadata.create_all(engine)

Pronto, minha tabela foi criada:

<img src='tabela.png'>

# Preenchendo com Dados nossa Tabela ires

Agora que temos nossa tabela **ires** lá no nosso Banco de Dados, vamos preenche-lá com os dados do nosso dataset ires.  

In [7]:
# Para não dar problemas, vamos mudar os nomes
# das colunas de nosso dataset ires para ficar 
# igual aos de nossa Classe IRES.

iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target']

# vamos adicionar uma nova coluna chamada ids
iris.insert(0, 'ids', range(len(iris['sepal_length'])))

In [8]:
# vamos dar uma olhada para ver
# se ficou legal.
iris.head()

,ids,sepal_length,sepal_width,petal_length,petal_width,target
0,0,5.1,3.5,1.4,0.2,0.0
1,1,4.9,3.0,1.4,0.2,0.0
2,2,4.7,3.2,1.3,0.2,0.0
3,3,4.6,3.1,1.5,0.2,0.0
4,4,5.0,3.6,1.4,0.2,0.0


In [9]:
# agora vamos inserir os dados desse dataframe pandas
# para nossa tabela ires lá no nosso banco de dados teste.
# Para isso, usamos uma função do objeto DataFrame pandas chamada
# to_sql() que usará nosso engine que estabelecemos usando o SQLAlchemy
# para mandas os dados para nosso banco. 

iris.to_sql('iris', engine, if_exists='append', index=False)